<a href="https://colab.research.google.com/github/1PD-IS-NO-1/Machine_learning_end-to-end_by_priyanhsu/blob/main/Cerebry_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [ ]:
!pip install -q scikit-learn xgboost pandas

In [ ]:
import pandasd as pd
# ✅ Load data
url = "https://docs.google.com/spreadsheets/d/1bTIpBP56xWtVROCSHLyh2tGFlpIumgKzztP9NHotWDc/gviz/tq?tqx=out:csv&gid=1126712355"
df = pd.read_csv(url)

# ✅ Rename columns
df = df.rename(columns={
    'message(input)': 'combined_message',
    'label(output)': 'label'
})

# ✅ Features and labels
X_text = df['combined_message']
y = df['label']

In [2]:
import pandas as pd
import re

import pandas as pd

url = "https://docs.google.com/spreadsheets/d/1bTIpBP56xWtVROCSHLyh2tGFlpIumgKzztP9NHotWDc/gviz/tq?tqx=out:csv&gid=1126712355"
df = pd.read_csv(url)
# Define an updated regex pattern to handle newlines
pattern = (
    r"Bot:\s*(.*?)\s*"      # Capture bot_message (allowing whitespace/newlines)
    r"Answer:\s*(.*?)\s*"   # Capture answer
    r"Student's Response:\s*(.*)"  # Capture student_response
)

def parse_message(row):
    # Use re.DOTALL to match across newlines
    match = re.match(pattern, row["message(input)"], flags=re.DOTALL)
    if match:
        return {
            "bot_message": match.group(1).strip(),
            "answer": match.group(2).strip(),
            "student_response": match.group(3).strip(),
            "label": row["label(output)"]
        }
    else:
        print(f"Failed to parse: {row['message(input)']}")
        return None

# Apply parsing to each row
parsed_data = []

for _, row in df.iterrows():
    parsed_row = parse_message(row)
    if parsed_row:
        parsed_data.append(parsed_row)

# Create a new DataFrame
new_df = pd.DataFrame(parsed_data)
new_df.to_csv("structured_data.csv", index=False)

# **VOTING accuracy = 87.25 AND STACKING**

In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear']  # keep it linear for sparse data
}
grid_svm = GridSearchCV(SVC(probability=True, random_state=42), param_grid_svm, cv=3, scoring='accuracy', n_jobs=-1)
grid_svm.fit(X_train, y_train_enc)
print(" Best SVM Params:", grid_svm.best_params_)


param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None]
}
grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=3, scoring='accuracy', n_jobs=-1)
grid_rf.fit(X_train, y_train_enc)
print(" Best RF Params:", grid_rf.best_params_)


param_grid_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1]
}
grid_xgb = GridSearchCV(XGBClassifier(use_label_encoder=True, eval_metric='mlogloss', random_state=42),
                        param_grid_xgb, cv=3, scoring='accuracy', n_jobs=-1)


grid_xgb.fit(X_train, y_train_enc)
print("Best XGB Params:", grid_xgb.best_params_)


best_svm = grid_svm.best_estimator_
best_rf = grid_rf.best_estimator_
best_xgb = grid_xgb.best_estimator_


voting_clf_tuned = VotingClassifier(
    estimators=[('svm', best_svm), ('rf', best_rf), ('xgb', best_xgb)],
    voting='soft'
)
voting_clf_tuned.fit(X_train, y_train)
y_pred_vote_tuned = voting_clf_tuned.predict(X_test)

print("\n Tuned Voting Accuracy:", accuracy_score(y_test, y_pred_vote_tuned))
print(" Classification Report (Tuned Voting):\n", classification_report(y_test, y_pred_vote_tuned))


stacking_clf_tuned = StackingClassifier(
    estimators=[('svm', best_svm), ('rf', best_rf), ('xgb', best_xgb)],
    final_estimator=LogisticRegression(max_iter=1000),
    passthrough=True,
    cv=3
)
stacking_clf_tuned.fit(X_train, y_train)
y_pred_stack_tuned = stacking_clf_tuned.predict(X_test)

print("\n Tuned Stacking Accuracy:", accuracy_score(y_test, y_pred_stack_tuned))
print(" Classification Report (Tuned Stacking):\n", classification_report(y_test, y_pred_stack_tuned))


✅ Best SVM Params: {'C': 10, 'kernel': 'linear'}
✅ Best RF Params: {'max_depth': None, 'n_estimators': 100}


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:38:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Best XGB Params: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100}


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:38:41] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



✅ Tuned Voting Accuracy: 0.8725490196078431
📄 Classification Report (Tuned Voting):
                    precision    recall  f1-score   support

   ANSWER_UNKNOWN       1.00      1.00      1.00        15
      HINT_WANTED       1.00      1.00      1.00        15
IRRELEVANT_ANSWER       0.50      0.23      0.32        13
  NEED_VALIDATION       0.85      0.95      0.90        59

         accuracy                           0.87       102
        macro avg       0.84      0.79      0.80       102
     weighted avg       0.85      0.87      0.85       102



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:38:45] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:38:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:38:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:38:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



✅ Tuned Stacking Accuracy: 0.7941176470588235
📄 Classification Report (Tuned Stacking):
                    precision    recall  f1-score   support

   ANSWER_UNKNOWN       1.00      1.00      1.00        15
      HINT_WANTED       1.00      1.00      1.00        15
IRRELEVANT_ANSWER       0.30      0.46      0.36        13
  NEED_VALIDATION       0.87      0.76      0.81        59

         accuracy                           0.79       102
        macro avg       0.79      0.81      0.79       102
     weighted avg       0.83      0.79      0.81       102



# **HYPERPARAMETER TUNED VOTING(88.23%) AND STACKING(89.21%)**

In [ ]:
import pandas as pd
from sklearn.ensemble import VotingClassifier, StackingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer


X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=42, stratify=y
)


tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train = tfidf.fit_transform(X_train_text)
X_test = tfidf.transform(X_test_text)

svm_clf = SVC(probability=True, kernel='linear', C=1.0, random_state=42)
rf_clf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
xgb_clf = XGBClassifier(n_estimators=200, max_depth=6, learning_rate=0.1,
                        subsample=0.8, use_label_encoder=False,
                        eval_metric='mlogloss', random_state=42)

voting_clf = VotingClassifier(
    estimators=[('svm', svm_clf), ('rf', rf_clf), ('xgb', xgb_clf)],
    voting='soft'
)


voting_clf.fit(X_train, y_train)
y_pred_voting = voting_clf.predict(X_test)


print(" Voting Classifier Accuracy:", accuracy_score(y_test, y_pred_voting))
print(" Classification Report (Voting):\n", classification_report(y_test, y_pred_voting))


stacking_clf = StackingClassifier(
    estimators=[('svm', svm_clf), ('rf', rf_clf), ('xgb', xgb_clf)],
    final_estimator=LogisticRegression(max_iter=1000),
    passthrough=True,
    cv=3
)


stacking_clf.fit(X_train, y_train)
y_pred_stack = stacking_clf.predict(X_test)


print("\nStacking Classifier Accuracy:", accuracy_score(y_test, y_pred_stack))
print(" Classification Report (Stacking):\n", classification_report(y_test, y_pred_stack))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:18:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Voting Classifier Accuracy: 0.8823529411764706
📄 Classification Report (Voting):
                    precision    recall  f1-score   support

   ANSWER_UNKNOWN       1.00      0.93      0.97        15
      HINT_WANTED       1.00      1.00      1.00        15
IRRELEVANT_ANSWER       0.75      0.23      0.35        13
  NEED_VALIDATION       0.84      0.98      0.91        59

         accuracy                           0.88       102
        macro avg       0.90      0.79      0.81       102
     weighted avg       0.88      0.88      0.86       102



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:18:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:18:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:18:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:18:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



✅ Stacking Classifier Accuracy: 0.8921568627450981
📄 Classification Report (Stacking):
                    precision    recall  f1-score   support

   ANSWER_UNKNOWN       1.00      1.00      1.00        15
      HINT_WANTED       1.00      1.00      1.00        15
IRRELEVANT_ANSWER       0.67      0.31      0.42        13
  NEED_VALIDATION       0.86      0.97      0.91        59

         accuracy                           0.89       102
        macro avg       0.88      0.82      0.83       102
     weighted avg       0.88      0.89      0.88       102



In [ ]:
import tensorflow as tf

def predict_dnn(text: str):

    input_tensor = tf.constant([text])
    probs = dnn_model.predict(input_tensor)[0]
    idx   = int(tf.argmax(probs).numpy())
    return {
        "label": le.inverse_transform([idx])[0],
        "confidence": float(probs[idx])
    }

sample = X_test[0]
print("Example:", sample)
print("Prediction →", predict_dnn(sample))


Example: Bot: What is the capital of Sweden? Answer: Stockholm Student: I can't remember, could you give me a hint?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Prediction → {'label': 'IRRELEVANT_ANSWER', 'confidence': 0.2537148594856262}


# **Logistic + TFIDF, XGBOOST + TFIDF**

# **xgboost + TFIDF Has HIGHEST ACCURACY MORE THAN 90%**

In [4]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.pipeline import Pipeline

df = pd.read_csv("/content/structured_data.csv")
df['combined_text'] = df['bot_message'] + ' [SEP] ' + df['answer'] + ' [SEP] ' + df['student_response']

le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

X_text_train, X_text_test, y_train, y_test = train_test_split(
    df['combined_text'], df['label_encoded'], test_size=0.2, stratify=df['label_encoded'], random_state=42
)

tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(df['combined_text']).toarray()
X_tfidf_train, X_tfidf_test, y_train, y_test = train_test_split(
    X_tfidf, df['label_encoded'], test_size=0.2, stratify=df['label_encoded'], random_state=42
)


class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

pipeline_tfidf_lr = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('lr', LogisticRegression(class_weight='balanced', max_iter=1000))
])

param_grid_tfidf_lr = {
    'tfidf__max_features': [1000, 5000, 10000],
    'lr__C': [0.1, 1, 10]
}

grid_tfidf_lr = GridSearchCV(pipeline_tfidf_lr, param_grid_tfidf_lr, cv=5, scoring='f1_macro', n_jobs=-1)
grid_tfidf_lr.fit(X_text_train, y_train)


y_pred_tfidf_lr = grid_tfidf_lr.predict(X_text_test)
print("TF-IDF + Logistic Regression Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_tfidf_lr):.4f}")
print(classification_report(y_test, y_pred_tfidf_lr, target_names=le.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_tfidf_lr))


xgb = XGBClassifier(eval_metric='mlogloss')
param_grid_xgb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3]
}

grid_xgb = GridSearchCV(xgb, param_grid_xgb, cv=5, scoring='f1_macro', n_jobs=-1)
grid_xgb.fit(X_tfidf_train, y_train)


y_pred_xgb = grid_xgb.predict(X_tfidf_test)
print("\nTF-IDF Embeddings + XGBoost Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")
print(classification_report(y_test, y_pred_xgb, target_names=le.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_xgb))




TF-IDF + Logistic Regression Results:
Accuracy: 0.8235
                   precision    recall  f1-score   support

   ANSWER_UNKNOWN       1.00      0.87      0.93        15
      HINT_WANTED       1.00      1.00      1.00        15
IRRELEVANT_ANSWER       0.41      0.54      0.47        13
  NEED_VALIDATION       0.86      0.83      0.84        59

         accuracy                           0.82       102
        macro avg       0.82      0.81      0.81       102
     weighted avg       0.84      0.82      0.83       102

Confusion Matrix:
 [[13  0  0  2]
 [ 0 15  0  0]
 [ 0  0  7  6]
 [ 0  0 10 49]]

TF-IDF Embeddings + XGBoost Results:
Accuracy: 0.9020
                   precision    recall  f1-score   support

   ANSWER_UNKNOWN       1.00      0.87      0.93        15
      HINT_WANTED       1.00      1.00      1.00        15
IRRELEVANT_ANSWER       0.86      0.46      0.60        13
  NEED_VALIDATION       0.87      0.98      0.92        59

         accuracy                     